#### Ingestion del archivo "country.json"

In [0]:
dbutils.widgets.text("p_enviroment","")
v_enviroment = dbutils.widgets.get("p_enviroment")

In [0]:
dbutils.widgets.text("p_file_date","2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/common_functions"

In [0]:
%run "../includes/configuration"

##### Paso 1 - Leer el archivo JSON usando "DataframeReader" de Spark

In [0]:
##### Paso 1 - Leer el archivo JSON usando "DataframeReader" de Spark

countries_schema = "countryId INT, countryIsoCode STRING, countryName STRING"

In [0]:
countries_df = spark.read.schema(countries_schema).json(f"{bronze_folder_path}/{v_file_date}/country.json")

#### Paso 2 - Eliminar columnas no deseadas del DataFrame

In [0]:
countries_dropped_df = countries_df.drop("countryIsoCode")

#### Paso 3 - Cambiar el nombre de las columnas y añadir "ingestion_date" y "enviroment"

In [0]:
from pyspark.sql.functions import current_timestamp, lit

In [0]:
final_countries_df = add_ingestion_date(countries_dropped_df) \
                    .withColumnRenamed("countryId","country_id") \
                    .withColumnRenamed("countryName","country_name") \
                    .withColumn("enviroment",lit(v_enviroment)) \
                    .withColumn("file_date",lit(v_file_date))

#### Paso 4 - Carga del dataFrame en archivo Parquet

In [0]:
final_countries_df.write.mode("overwrite").format("delta").saveAsTable("movie_silver.country")

In [0]:
dbutils.notebook.exit("Succes!")